In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana-2.mwt2.org', 'port':9200}],timeout=60)

In [8]:
myquery = {
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "NOT produsername:gangarbt AND jobname:*",
          "analyze_wildcard": True,
          "lowercase_expanded_terms": False
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "query": "processingtype:reprocessing AND proddblock:*.00282712.* AND jobstatus:\"finished\" AND proddblock:\"data15_13TeV:data15_13TeV.00283429.physics_Main.daq.RAW\"",
                  "analyze_wildcard": True,
                  "lowercase_expanded_terms": False
                }
              }
            },
            {
              "range": {
                "modificationtime": {
                  "gte": 1455014531346,
                  "lte": 1456742531346,
                  "format": "epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      }
    }
  },
  "highlight": {
    "pre_tags": [
      "@kibana-highlighted-field@"
    ],
    "post_tags": [
      "@/kibana-highlighted-field@"
    ],
    "fields": {
      "*": {}
    },
    "require_field_match": False,
    "fragment_size": 2147483647
  },
  "size": 0,
  "aggs": {
    "2": {
      "histogram": {
        "script": "(doc['nevents'].value) ? doc['wall_time'].value*10/doc['nevents'].value : 0",
        "lang": "expression",
        "interval": 5
      },
      "aggs": {
        "3": {
          "significant_terms": {
            "field": "atlasrelease",
            "size": 10
          }
        }
      }
    }
  }
}

myquery2= {
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "NOT produsername:gangarbt AND jobname:*",
          "analyze_wildcard": True,
          "lowercase_expanded_terms": False
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "query": "processingtype:reprocessing AND proddblock:*.00282712.* AND jobstatus:\"finished\" AND proddblock:\"data15_13TeV:data15_13TeV.00283429.physics_Main.daq.RAW\"",
                  "analyze_wildcard": True,
                  "lowercase_expanded_terms": False
                }
              }
            },
            {
              "range": {
                "modificationtime": {
                  "gte": 1455014531346,
                  "lte": 1456742531346,
                  "format": "epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      }
    }
  },
  "highlight": {
    "pre_tags": [
      "@kibana-highlighted-field@"
    ],
    "post_tags": [
      "@/kibana-highlighted-field@"
    ],
    "fields": {
      "*": {}
    },
    "require_field_match": False,
    "fragment_size": 2147483647
  },
  "size": 0,
  "aggs": {
    "2": {
      "histogram": {
        "script": "doc['maxpss'].value*1024",
        "lang": "expression",
        "interval": 200000000
      },
      "aggs": {
        "3": {
          "significant_terms": {
            "field": "atlasrelease",
            "size": 10
          }
        }
      }
    }
  }
}

ind="jobs2016"

In [9]:
res= es.search( index=ind, body=myquery2, analyze_wildcard=True)

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='atlas-kibana-2.mwt2.org', port=9200): Read timed out. (read timeout=60))

In [ ]:
print(res)